# Assignment 2: Information Extraction

In [1]:
import nltk
import re

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Ger\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Ger\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\Ger\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\Ger\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\Ger\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\Ger\AppData

True

## Task 1: Named Entity Annotation (10 Marks)

Using the IOB tagging scheme annotate all of the named entities (PERson, LOCation, ORGanisation, TIME) in the following sentence:

*Wayne Rooney is a professional footballer from England who last played for Major League Soccer club D.C. United and will join Derby County in January 2020.*

Edit this cell and write your annotation below the line. (Note that you don't have to write code for this task, you have to annotate it manually)

---
('Wayne Rooney', 'B-PERS'), ('is','O-VB'), ('a','O-DT'), ('professional', 'O-JJ'), ('footballer', O-NN), ('from', 'O-IN'), 
('England', 'B-LOC'), ('who', 'O-WP'), ('last', 'O-JJ'), ('played' 'O-VBD'), ('for', 'O-IN'), 
('Major League Soccer', 'B-ORG'), ('club', 'O-NN'), 
('D.C. United', 'B-ORG'), ('and', 'O-CC'), ('will', 'O-MD'), ('join', 'O-VB'),
('Derby County', 'B-LOC'), ('in', 'O-IN'), ('January 2020', 'O-TIME'), ('.', 'O-.')


---

### For subsequent tasks in this assignment, you will work with the documents in `football_players.txt` to perform various information extraction tasks.

In [2]:
# Download the text file (uncomment the line below in this cell, if not already downloaded from Blackboard)
# !curl "https://ideone.com/plain/OvwDXZ" > football_players.txt

 Read all the documents from `football_players.txt` into a list called `docs`.

In [3]:
docs = []
# your code goes here
import csv
with open('football_players.txt', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter = '.')
    for i in reader:
        docs.append(i)

## Task 2 (10 Marks)
Write a function that takes a document and returns a list of sentences with part-of-speech tags.

Please keep in mind that the expected output is a list within a list as shown below.


Hint: For this task you need to perform three steps:
1. Sentence Segmentation
1. Word Tokenization
1. Part-of-Speech Tagging

In [4]:
from nltk import pos_tag, word_tokenize

def ie_preprocess(document):
    # your code goes here
    tagg_sent = []
    for i in document:
        lst = []
        token = word_tokenize(i)
        word_tag = pos_tag(token)
        for x, y in word_tag:
            lst.append(((x), (y)))
        tagg_sent.append(lst)
    return tagg_sent

Run the cell below to verify your result for the second sentence in the first document.
Expected output: 
`[('He', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('forward', 'NN'), ('and', 'CC'), ('serves', 'NNS'), ('as', 'IN'), ('captain', 'NN'), ('for', 'IN'), ('Portugal', 'NNP'), ('.', '.')]`

In [5]:
first_doc = docs[0]
tagged_sentences = ie_preprocess(first_doc)
tagged_sentences[1]

[('He', 'PRP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('forward', 'NN'),
 ('and', 'CC'),
 ('serves', 'NNS'),
 ('as', 'IN'),
 ('captain', 'NN'),
 ('for', 'IN'),
 ('Portugal', 'NNP')]

## Task 3 (20 Marks)
Write a function that takes a list of tokens with POS tags for a sentence and returns a list of named entities (NE). 

Hint: Use `binary = True` while calling NE chunk function

In [6]:
import nltk

def find_named_entities(sent):
    named_entities = []
    # your code goes here
    chunks = nltk.ne_chunk(sent, binary=True)
    items = nltk.chunk.tree2conlltags(chunks)
    for i in items:
        named_entities.append((i[0],i[2]))
    # code adapted from:
    # https://stackoverflow.com/questions/30664677/extract-list-of-persons-and-organizations-using-stanford-ner-tagger-in-nltk
    cont_chunk = []
    curr_chunk = []
    for token, tag in named_entities:
        if tag != 'O':
            curr_chunk.append((token, tag))
        else:
            if curr_chunk:
                cont_chunk.append(curr_chunk)
                curr_chunk = []
        if curr_chunk:
            cont_chunk.append(curr_chunk)
    n_e = [" ".join([token for token, tag in ne]) for ne in cont_chunk]
    nam_ent = set()
    named_entities = [i for i in n_e if not (i in nam_ent or nam_ent.add(i))] 
    return named_entities

Run the cell below to verify your result for the first sentence in the first document.
Expected output: `['Cristiano Ronaldo', 'Santos Aveiro', 'ComM', 'GOIH', 'Portuguese', 'Portuguese', 'Spanish', 'Real Madrid', 'Portugal']`

In [7]:
tagged_sentences = ie_preprocess(docs[0])
find_named_entities(tagged_sentences[0])

['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

## Task 4 (5 Marks)

Implement the `find_all_named_entities` function below to find **all** NEs in a given document.

Hint: Use `find_named_entities` implemented above for this task.

In [8]:
def find_all_named_entities(doc):
    named_entities = []
    # your code goes here
    if any(isinstance(i, list) for i in doc) == True:
        for i in doc:
            tagged_sent = ie_preprocess(i)
            for i in tagged_sent:
                ne = find_named_entities(i)
                named_entities.append(ne)
    
    else:
        tagged_sent = ie_preprocess(doc)
        for i in tagged_sent:
            ne = find_named_entities(i)
            named_entities.append(ne)
    
    named_entities = [i for j in named_entities for i in j]
    return named_entities   # return a flat list and not a list of lists

How many named entities did you find in the first document?

In [9]:
num_ne = find_all_named_entities(docs[0])
print(len(num_ne))

55


## Task 5 (5 Marks)

Find named entities across **all** documents in `football_players.txt`, and save the result into a single flat list.

In [10]:
all_named_entities = []
# your code goes here
for i in docs:
    tagged_sent = ie_preprocess(i)
    for i in tagged_sent:
        ne = find_named_entities(i)
        all_named_entities.append(ne)

all_named_entities = [i for j in all_named_entities for i in j]

How many named entities did you find across all documents?

In [11]:
print(len(all_named_entities))

356


## Task 6 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the `re.compile()` function to create the extraction patterns.

Reference: https://docs.python.org/3/howto/regex.html

In [12]:
import os
java_path = r"C:\Program Files\Java\jdk-15.0.1\bin\java.exe"
os.environ['JAVAHOME'] = java_path

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from collections import Counter

st = StanfordNERTagger(r'C:\Users\Ger\Desktop\College\CT5146\stanford-ner-4.2.0\stanford-ner-2020-11-17\classifiers\english.muc.7class.distsim.crf.ser.gz',
                       r'C:\Users\Ger\Desktop\College\CT5146\stanford-ner-4.2.0\stanford-ner-2020-11-17\stanford-ner.jar',
                        encoding = 'utf-8')

def name_of_the_player(doc):
    name = []
    n = []
    if any(isinstance(i, list) for i in doc) == True:
        for i in doc:
            for j in i:
                tokenized_text = word_tokenize(j)
                classified_text = st.tag(tokenized_text)
                people = [i for i in classified_text if i[1]=='PERSON']
                if len(people) > 0: #take full names, i.e. two name elements as a minimum for full names
                    n.append(' '.join([i[0] for i in people])) ##
    
        freq = Counter(n)
        most_freq = freq.most_common(1)[0]
       
        for i in n:
            j = str(i).split() # full name split
            if len(j) > 1:
                if j[0] == most_freq[0]: # to ensure we are not just referencing the first or last name
                    pass
                else:
                    for a, b in enumerate(j):
                        if most_freq[0] in b and len(b) > 1:
                            name.append((' '.join((j[a-1],j[a])))) 
                
            return list(set(name)) ##
    
    else:
        for i in doc:
            j = str(i)
            tokenized_text = word_tokenize(j)
            classified_text = st.tag(tokenized_text)
            people = [i for i in classified_text if i[1]=='PERSON']
            if len(people) > 0: #take full names, i.e. two name elements as a minimum for full names
                n.append(' '.join([i[0] for i in people])) ##
    
        freq = Counter(n)
        most_freq = freq.most_common(1)[0]
    
        for i in n:
            j = i.split()
            if most_freq[0] in j and len(j) > 1:
                name.append(i)
                
        return list(name) #
    
def country_of_origin(doc):
    player = name_of_the_player(doc)
    if any(isinstance(i, list) for i in doc) == True:
        tagged_words = []
        for i in doc:
            for j in i:
                tokenized_text = word_tokenize(j)
                classified_text = st.tag(tokenized_text)
                if len(classified_text) > 0:
                    tagged_words.append(classified_text)
                else:
                    pass

        root1 = []
        for i in tagged_words:
            for j in i:
                # not great only identifies languages/nationality of countries with these endings
                if j[0][-3:] in ('ish','ese','ine','ian'):
                    word_root = j[0][:4]
                    word = j[0][0]
                    root1.append((word_root, word))

        root2 = []
        for i in tagged_words:
            for j in i:
                place = [x for x in j if j[1]=='LOCATION']
                if len(place) > 0 and len(place[0]) > 4:
                    word_root = place[0][:4]
                    word = place[0]
                    root2.append((word_root, word))
    
        cntry = []
        for i in root1:
            for j in root2:
                if i[0] == j[0]:
                    cntry.append(j[1])
        
        freq = Counter(cntry)
        country = freq.most_common(1)[0]
        
        return ('Player: '+ str(player) + ' Country: '+ str(country))
                                   
    else:
        tagged_words = []
        for i in doc:
            j = str(i)
            tokenized_text = word_tokenize(j)
            classified_text = st.tag(tokenized_text)
            if len(classified_text) > 0:
                tagged_words.append(classified_text)
            else:
                pass

        root1 = []
        for i in tagged_words:
            for j in i:
            # not great only identifies languages/nationality of countries with these endings
            # captures English/British, Portuguese, Argentine/Argentinian (not Spanish)
            # could potentially add an edit distance to get Spain -> Spanish
                if j[0][-3:] in ('ish','ese','ine','ian'):
                    word_root = j[0][:4]
                    word = j[0]
                    root1.append((word_root, word))

        root2 = []
        for i in tagged_words:
            for j in i:
                place = [x for x in j if j[1]=='LOCATION']
                if len(place) > 0 and len(place[0]) > 4:
                    word_root = place[0][:4]
                    word = place[0]
                    root2.append((word_root, word))
    
        cntry = []
        for i in root1:
            for j in root2:
                if i[0] == j[0]:
                    cntry.append(j[1])
        
        freq = Counter(cntry)
        country = freq.most_common(1)[0]
        
        return ('Player: '+ str(player[0]) + ' Country: '+ str(country[0]))

def date_of_birth(doc):
    player = name_of_the_player(doc)
    if any(isinstance(i, list) for i in doc) == True:
        for i in doc:
            for j in i:
                match_re = re.compile('[0-9]+\s[A-Z][a-z]+\s[0-9]+')
                date_str = re.findall(match_re, j)
                if len(date_str) > 0:
            
                    return ('Player: '+ str(player) + ' DOB: '+ str(date_str))

    else:
        for i in doc:
            j = str(i)
            match_re = re.compile('[0-9]+\s[A-Z][a-z]+\s[0-9]+')
            date_str = re.findall(match_re, j)
            if len(date_str) > 0:
            
                return ('Player: '+ str(player) + ' DOB: '+ str(date_str))

def team_of_player(doc):
    lst = []
    player = name_of_the_player(doc)
    for i in doc:
        j = str(i).split()
        for i in j:
            k = i.strip('"()[];,')
            reg_exp = re.compile('[A-Za-z]+')
            match_re = re.findall(reg_exp,k)
            if len(match_re) == 1:
                lst.append(match_re)

    lst1 = [i for j in lst for i in j]
    lst2 = (' '.join(lst1)).split('club',1)[1]
    token_words = word_tokenize(lst2)
    sent = ie_preprocess(token_words)

    named_entities = []
    for i in sent:
        chunks = nltk.ne_chunk(i, binary=True)
        items = nltk.chunk.tree2conlltags(chunks)
        for i in items:
            named_entities.append((i[0],i[2]))

    cont_chunk = []
    curr_chunk = []
    for token, tag in named_entities:
        if tag != 'O':
            curr_chunk.append((token, tag))
        else:
            if curr_chunk:
                cont_chunk.append(curr_chunk)
                curr_chunk = []
        if curr_chunk:
            cont_chunk.append(curr_chunk)
    n_e = [" ".join([token for token, tag in ne]) for ne in cont_chunk]
    nam_ent = set()
    named_entities = [i for i in n_e if not (i in nam_ent or nam_ent.add(i))] 
    
    return('Player: '+ str(player) + ' Team: '+ str(named_entities))

#def position_of_the_player(doc): -- Not attempted as I have to move on to next assignment
#  # your code goes here
#  return position

Execute the cell below to verify the `date_of_birth` function for the third player. Expected output `5 February 1992`


In [13]:
date_of_birth(docs[2])

"Player: [] DOB: ['5 February 1992']"

## Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract it.

In [14]:
# bit of a cheat as its just the same code block I used above except swapping our country for language
def language_of_the_player(doc):
    player = name_of_the_player(doc)
    tagged_words = []
    if any(isinstance(i, list) for i in doc) == True:
        for i in doc:
            for j in i:
                tokenized_text = word_tokenize(j)
                classified_text = st.tag(tokenized_text)
                if len(classified_text) > 0:
                    tagged_words.append(classified_text)
                else:
                    pass

        root1 = []
        for i in tagged_words:
            for j in i:
                # not great only identifies languages/nationality of countries with these endings
                if j[0][-3:] in ('ish','ese','ine','ian'):
                    word_root = j[0][:4]
                    word = j[0][0]
                    root1.append((word_root, word))
                    
        root2 = []
        for i in tagged_words:
            for j in i:
                place = [x for x in j if j[1]=='LOCATION']
                if len(place) > 0 and len(place[0]) > 4:
                    word_root = place[0][:4]
                    word = place[0]
                    root2.append((word_root, word))
    
        lang = []
        for i in root1:
            for j in root2:
                if i[0] == j[0]:
                    lang.append(i[1])
        
        freq = Counter(lang)
        language = freq.most_common(1)[0]
        
        return ('Player:' + str(player) + 'Lang: ' + str(language))
    
    else:
        for i in doc:
            j = str(i)
            tokenized_text = word_tokenize(j)
            classified_text = st.tag(tokenized_text)
            if len(classified_text) > 0:
                tagged_words.append(classified_text)
            else:
                pass
        
        root1 = []
        for i in tagged_words:
            for j in i:
                if j[0][-3:] in ('ish','ese','ine','ian','ina'):
                    word_root = j[0][:4]
                    word = j[0]
                    root1.append((word_root,word))
        
        root2 = []
        for i in tagged_words:
            for j in i:
                place = [x for x in j if j[1]=='LOCATION']
                if len(place) > 0 and len(place[0]) > 4:
                    word_root = place[0][:4]
                    word = place[0]
                    root2.append((word_root, word))
    
        lang = []
        for i in root1:
            for j in root2:
                if i[0] == j[0]:
                    lang.append(i[1])
        
        freq = Counter(lang)
        language = freq.most_common(1)[0]
        
        return ('Player:' + str(player) + ' Lang: ' + str(language))
    
print(language_of_the_player(docs[0]))

Player:['Cristiano Ronaldo dos Santos Aveiro'] Lang: ('Portuguese', 40)
